# Asynchronous and concurrent execution on GPUs
>*Melina Abeling, Julian Aeissen, Michele Pagani*. Supervised by *Oliver Fuhrer*

*GPUs allow for asynchronous (CPU simply launches work on GPU and then continues) and concurrent (multiple tasks are executed in parallel on the GPU) execution. In this project you will implement a simple stencil program and investigate performance using different stencil motifs as a function of grid size and amount of concurrency. It is foreseen to use either CuPy or CUDA for this project.*


---

## Introduction
*TODO*

---

## Methods
*TODO*

In [1]:
# Imports
import timeit
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

In [15]:
# Utils
def func_time(title="DEFAULT"):
    def decorator(func):
        def wrapper(*args, **kwargs):
            tic = timeit.default_timer()
            res = func(*args, **kwargs)
            toc = timeit.default_timer()
            print(f"[{title}] Time: {toc - tic} s")
            return res
        return wrapper
    return decorator

### Stencils
*TODO*

#### A
*TODO*

In [19]:
def stencilA():
    pass

#### B
*TODO*

In [5]:
def stencilB():
    pass

### Sequential
*TODO*

In [ ]:
@func_time("Sequential")
def sequential_computation(stencil, field):
    pass

### Concurrent
*TODO*

In [20]:
@func_time("Parallel")
def parallel_computation(stencil, field, n_stream):
    pass

### Initial field
*TODO*

In [21]:
def get_initial_field(h, w):
    pass

---

## Results
*TODO*

---

## Discussion
*TODO*

---

## Conclusion
*TODO*